In [1]:
#Imprting Libraries
import ast
import glob
import re
from pathlib import Path

import astor
import pandas as pd
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
# #pip install nmslib
#import nmslib
import numpy
import re
#python -m spacy download en_core_web_sm
EN = spacy.load('en_core_web_sm')

In [ ]:
#Downloading the dataset
# ! wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
# ! unzip python.zip

In [2]:
#Reading and Parsing data from database
path = Path('.')
#collapse_show
def jsonl_list_to_dataframe(file_list, columns=None):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

def get_dfs(path):
    """Grabs the different data splits and converts them into dataframes"""
    dfs = []
    for split in ["train", "valid", "test"]:
        files = sorted((path/split).glob("**/*.gz"))
        df = jsonl_list_to_dataframe(files, ["code", "docstring","code_tokens","docstring_tokens","url"])
        dfs.append(df)
        
    return dfs


def tokenize_docstring(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]


def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

df_trn, df_val, df_tst = get_dfs(path/"python/final/jsonl")


In [3]:
#Subsampling the original database
df_trn_sub = df_trn[:5000]
print(df_trn_sub.shape)

(5000, 5)


In [4]:
docstring = list(df_trn_sub['docstring'].values)
code = list(df_trn_sub['code'].values)
#print(code[0])

In [5]:
codeToken = []
docToken = []

for singleCode,singleDoc in tqdm(zip(code,docstring)):
  codeToken.append(' '.join(tokenize_code(singleCode)))
  docToken.append(' '.join(tokenize_docstring(singleDoc)))

5000it [00:07, 685.75it/s] 


In [6]:
df_trn_sub['code token'] = codeToken
df_trn_sub['doc token'] = docToken

/var/folders/b0/nlrr4t_x3j5_jvwwktffc3j00000gn/T/ipykernel_54834/2253202713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trn_sub['code token'] = codeToken
/var/folders/b0/nlrr4t_x3j5_jvwwktffc3j00000gn/T/ipykernel_54834/2253202713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trn_sub['doc token'] = docToken


In [10]:
#Clean Code Token
code_token = list(df_trn_sub['code_tokens'].values)
doc_token = list(df_trn_sub['doc token'].values)
code_token_list = []
doc_token_list = []
for code,doc in tqdm(zip(code_token,doc_token)):
    code_token_list.append(' '.join(code))
    doc_token_list.append(' '.join(doc))

import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf = pd.DataFrame()
df_idf['text'] = code_token_list
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

df_idf['doc text'] = doc_token_list
df_idf['doc text'] = df_idf['doc text'].apply(lambda x:pre_process(x))

clean_code = list(df_idf['text'].values)
clean_doc = list(df_idf['doc text'].values)
clean_code_token = []
for code,doc in zip(clean_code,clean_doc):
  clean_code_token.append(code.split(' ')+doc.split(' '))
df_trn_sub['clean_code'] = clean_code_token

5000it [00:00, 5610.78it/s]
/var/folders/b0/nlrr4t_x3j5_jvwwktffc3j00000gn/T/ipykernel_54834/49740048.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trn_sub['clean_code'] = clean_code_token


In [11]:
df_trn_sub.head()

,code,docstring,code_tokens,docstring_tokens,url,code token,doc token,clean_code
0,"def train(train_dir, model_save_path=None, n_n...",Trains a k-nearest neighbors classifier for fa...,"[def, train, (, train_dir, ,, model_save_path,...","[Trains, a, k, -, nearest, neighbors, classifi...",https://github.com/ageitgey/face_recognition/b...,def train train_dir model_save_path None n_nei...,trains a k - nearest neighbors classifier for ...,"[def, train, train_dir, model_save_path, none,..."
1,"def predict(X_img_path, knn_clf=None, model_pa...",Recognizes faces in given image using a traine...,"[def, predict, (, X_img_path, ,, knn_clf, =, N...","[Recognizes, faces, in, given, image, using, a...",https://github.com/ageitgey/face_recognition/b...,def predict X_img_path knn_clf None model_path...,recognizes faces in given image using a traine...,"[def, predict, x_img_path, knn_clf, none, mode..."
2,"def show_prediction_labels_on_image(img_path, ...",Shows the face recognition results visually.\n...,"[def, show_prediction_labels_on_image, (, img_...","[Shows, the, face, recognition, results, visua...",https://github.com/ageitgey/face_recognition/b...,def show_prediction_labels_on_image img_path p...,shows the face recognition results visually . ...,"[def, show_prediction_labels_on_image, img_pat..."
3,"def _rect_to_css(rect):\n """"""\n Convert ...",Convert a dlib 'rect' object to a plain tuple ...,"[def, _rect_to_css, (, rect, ), :, return, rec...","[Convert, a, dlib, rect, object, to, a, plain,...",https://github.com/ageitgey/face_recognition/b...,def _rect_to_css rect Convert a dlib rect obje...,convert a dlib ' rect ' object to a plain tupl...,"[def, _rect_to_css, rect, return, rect, top, r..."
4,"def _trim_css_to_bounds(css, image_shape):\n ...","Make sure a tuple in (top, right, bottom, left...","[def, _trim_css_to_bounds, (, css, ,, image_sh...","[Make, sure, a, tuple, in, (, top, right, bott...",https://github.com/ageitgey/face_recognition/b...,def _trim_css_to_bounds css image_shape Make s...,"make sure a tuple in ( top , right , bottom , ...","[def, _trim_css_to_bounds, css, image_shape, r..."


In [12]:

#Original Tokens
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
data_corpus = list(df_trn_sub.clean_code.values)
document_corpus = list(df_trn_sub.docstring.values)
buffer = []
for data in data_corpus:buffer.append(' '.join(data))
vocabulary=vectorizer.fit(buffer)
X= vectorizer.transform(buffer)
# print('Code Tokens : ',vocabulary.get_feature_names())
#print('Word Embeddings : ',X.toarray())
baseData = X.toarray()

In [13]:
import numpy as np
from scipy.spatial import distance
from sklearn.metrics import ndcg_score
from tqdm import tqdm
testDocstring = list(df_tst.docstring.values)
print(len(testDocstring))
filteredDocstring = []
for doc in testDocstring:
    buffer = doc.split(' ')
#   if len(buffer) < 10:filteredDocstring.append(doc)
    filteredDocstring.append(doc)
filteredDocstring = filteredDocstring[:400]


22176


In [14]:

from sklearn.metrics.pairwise import cosine_similarity
closest_doc_indexes = {}
testingVector = vectorizer.transform(filteredDocstring)
doctrainingVector = vectorizer.transform(document_corpus)
for index, test_doc_embd in tqdm(enumerate(testingVector)):
  test_doc_embd = np.reshape(test_doc_embd,(1,-1))
  similarities = cosine_similarity(test_doc_embd,doctrainingVector)[0]
  
  most_similar_train_doc = np.argmax(similarities)
  closest_doc_indexes[index] = most_similar_train_doc

#print(closest_doc_indexes)


400it [00:01, 352.99it/s]


In [15]:

result = []
ndcga = []
mrr = []
for doc in tqdm(filteredDocstring):
    testingString = [doc]
    testingVector = vectorizer.transform(testingString)
    testingVector = testingVector.toarray()
    distances = distance.cdist(testingVector, baseData, "cosine")[0]
    min_index = np.argmin(distances)
    min_distance = distances[min_index]
    max_similarity = 1 - min_distance
    result.append(max_similarity)
    ndcga.append((ndcg_score(np.asarray(testingVector), np.asarray([baseData[min_index]]))))
    index = filteredDocstring.index(doc)
    closest = closest_doc_indexes[index]
    best = np.argsort(distances)[:50]
    
    if closest in best:
      best = list(best)
      mrr.append(1/(best.index(closest)+1))
    else:
      mrr.append(0)
  

100%|█████████████████████████████████████████| 400/400 [48:29<00:00,  7.27s/it]


In [16]:
result = [x for x in result if str(x) != 'nan']

print('NDCGA Results : ',sum(ndcga)/len(ndcga))
print('Similarity Average : ',sum(result)/len(result))
print('Mean Reciprocal Rank :  ',sum(mrr)/len(mrr))

0.543236839661495
0.3966358238222773
0.0224011853126206
